In [1]:
import numpy as np
import networkx as nx
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number
V=10000

In [3]:
file_name='full_graph_{}_{}.txt'.format(n,N)
file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)

# Reading graph

In [4]:
con=[[] for _ in range(n+N+1)]
edges=set()

In [5]:
with open('test/'+file_name, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        con[x].append((y,w))
        con[y].append((x,w))
        edges.add((x,y,w))

           
    f.close()

In [6]:
#con
edges

{(1, 11, 0.8550095815477146),
 (1, 13, 0.13651902276258543),
 (1, 14, 0.4552114523664397),
 (1, 15, 0.7193580942721072),
 (2, 3, 0.3358722859296459),
 (2, 4, 0.8599331292728976),
 (2, 5, 0.24832691505262272),
 (2, 6, 0.4303409705286523),
 (2, 7, 0.8600341979033854),
 (2, 8, 0.5710015751835147),
 (2, 9, 0.24209810441616286),
 (2, 10, 0.9842410387580762),
 (2, 11, 0.9559853575911531),
 (2, 12, 0.38665511612196435),
 (2, 13, 0.7675423843816244),
 (3, 2, 0.3358722859296459),
 (3, 4, 0.3671880920795716),
 (3, 7, 0.6877173709908387),
 (3, 9, 0.33828160798887774),
 (3, 13, 0.7211358616091744),
 (3, 14, 0.6729130828832756),
 (3, 15, 0.7713415032141654),
 (4, 2, 0.8599331292728976),
 (4, 3, 0.3671880920795716),
 (4, 5, 0.2688155114523566),
 (4, 6, 0.10372897412869164),
 (4, 7, 0.8126127174604049),
 (4, 8, 0.9899069494890694),
 (4, 9, 0.7434852828685689),
 (4, 10, 0.9561471020946433),
 (4, 11, 0.574548906236377),
 (4, 12, 0.6756606494147235),
 (4, 13, 0.8178947166091707),
 (5, 2, 0.2483269150526

In [7]:
con[1]

[(11, 0.8550095815477146),
 (11, 0.8550095815477146),
 (13, 0.13651902276258543),
 (13, 0.13651902276258543),
 (14, 0.4552114523664397),
 (14, 0.4552114523664397),
 (15, 0.7193580942721072),
 (15, 0.7193580942721072)]

# Reading shortest paths

In [8]:
sh_path=[[] for _ in range(n+N+1)]

for i in range(n+N+1):
    sh_path[i]=[[] for _ in range(n+N+1)]

In [9]:
with open('test/'+file_name_shortest_path, 'r') as f:
    
    n,N=map(int,f.readline().split())
    
    for i in range(1,n+N+1):
        for j in range(1,n+N+1):
            if i!=j:
                x,y=map(int,f.readline().split())
                #print('x,y',x,y)
                
                while True:
                    line1=f.readline()
                    #print('line1',line1)
                    
                    if not line1.strip():
                        break

                    line2=f.readline()
                    
                    w=float(line1)
                    path=list(map(int,line2.split()))
                    #print('path',path)
                    #print('w',w)

                    sh_path[i][j].append((w,path))


In [10]:
sh_path[1]

[[],
 [],
 [(0.8553214489761374, [1, 13, 9, 2]),
  (0.9040614071442098, [1, 13, 2]),
  (1.1296863283489438, [1, 15, 5, 2])],
 [(0.8576548843717599, [1, 13, 3]),
  (0.9515049525488523, [1, 13, 9, 3]),
  (1.1281245352497153, [1, 14, 3])],
 [(0.9544137393717561, [1, 13, 4]),
  (1.1501749247486777, [1, 15, 5, 4]),
  (1.1562559524775953, [1, 15, 6, 4])],
 [(0.8813594132963211, [1, 15, 5]),
  (0.9231407700419829, [1, 15, 11, 5]),
  (0.9698721415136141, [1, 11, 5])],
 [(1.0525269783489035, [1, 15, 6]),
  (1.0581427135004478, [1, 13, 4, 6]),
  (1.2539038988773692, [1, 15, 5, 4, 6])],
 [(0.6312351366215551, [1, 13, 9, 7]),
  (0.9163131002561277, [1, 13, 7]),
  (1.0280404240483327, [1, 14, 7])],
 [(0.7617558106030508, [1, 13, 9, 7, 8]),
  (1.0468337742376232, [1, 13, 7, 8]),
  (1.1585610980298284, [1, 14, 7, 8])],
 [(0.6132233445599745, [1, 13, 9]),
  (0.9343248923177082, [1, 13, 7, 9]),
  (1.0460522161099133, [1, 14, 7, 9])],
 [(1.0319904351418017, [1, 13, 9, 7, 8, 10]),
  (1.0945587965603745, 

# Figuring out the coefficients gamma

In [11]:
edges_to_path=set()

In [12]:
def belong_to_path():
   
    for i in range(1,N+1):
        for j in range(N+1,n+N+1):

            #print('i,j,',i,j)
            path=sh_path[i][j][0][1]
            l=len(path)

            for k in range(l-1):

                #if path[k]>n and path[k+1]>n:
                #    continue

                edges_to_path.add((path[k],path[k+1],i,j))

In [13]:
belong_to_path()

In [14]:
sh_path[1][2][0][1]

[1, 13, 9, 2]

In [15]:
edges_to_path

{(1, 13, 1, 12),
 (1, 13, 1, 13),
 (1, 14, 1, 14),
 (1, 15, 1, 11),
 (1, 15, 1, 15),
 (2, 5, 2, 11),
 (2, 5, 2, 15),
 (2, 9, 2, 13),
 (2, 9, 2, 14),
 (2, 12, 1, 12),
 (2, 12, 2, 12),
 (2, 12, 3, 12),
 (2, 12, 5, 12),
 (2, 12, 7, 12),
 (2, 12, 8, 12),
 (2, 12, 9, 12),
 (2, 12, 10, 12),
 (3, 2, 3, 12),
 (3, 9, 3, 11),
 (3, 9, 3, 15),
 (3, 13, 3, 13),
 (3, 14, 3, 14),
 (3, 14, 4, 14),
 (3, 14, 6, 14),
 (4, 3, 4, 14),
 (4, 3, 6, 14),
 (4, 5, 4, 11),
 (4, 5, 4, 15),
 (4, 12, 4, 12),
 (4, 13, 4, 13),
 (4, 13, 6, 13),
 (5, 2, 5, 12),
 (5, 2, 10, 12),
 (5, 11, 2, 11),
 (5, 11, 4, 11),
 (5, 11, 5, 11),
 (5, 11, 5, 14),
 (5, 11, 10, 11),
 (5, 13, 5, 13),
 (5, 15, 2, 15),
 (5, 15, 4, 15),
 (5, 15, 5, 15),
 (5, 15, 10, 15),
 (6, 4, 6, 13),
 (6, 4, 6, 14),
 (6, 12, 6, 12),
 (6, 15, 6, 11),
 (6, 15, 6, 15),
 (7, 9, 7, 11),
 (7, 9, 7, 12),
 (7, 9, 7, 13),
 (7, 9, 7, 15),
 (7, 9, 8, 11),
 (7, 9, 8, 12),
 (7, 9, 8, 13),
 (7, 9, 8, 15),
 (7, 9, 10, 13),
 (7, 14, 2, 14),
 (7, 14, 5, 14),
 (7, 14, 7, 14),

# Collecting terms

In [16]:
def mult(a):
    d={}
    
    for q in a:
        for e in q['term']:
            for e2 in q['term']:
                
                x,y=e
                z,t=e2
                
                if x<z:
                    x,z=z,x
                    y,t=t,y
                
                coeff=q['coeff']
                            
                if x!='C' and z!='C':
                    d[(x,z)]=d[(x,z)]+coeff*y*t if (x,z) in d else coeff*y*t
                    
                if x=='C' and z!='C':
                    d[(z,z)]=d[(z,z)]+coeff*y*t if (z,z) in d else coeff*y*t
                    
                if x!='C' and z=='C':
                    d[(x,x)]=d[(x,x)]+coeff*y*t if (x,x) in d else coeff*y*t
                    
        #print(d)
        

                    
    return d

# Example

In [17]:
form=[{'coeff':1, 'term':[('y2',1)]}, {'coeff':4, 'term':[('x1',1), ('x2',1), ('y2',1)]}, {'coeff':1, 'term':[('x2',1)]}, {'coeff':1, 'term':[('y1',1)]}, 
      {'coeff':1, 'term':[('x1',1), ('y2',1)]}, {'coeff':9, 'term':[('x1',1), ('y1',1)]}, {'coeff':25, 'term':[('x1',1), ('x2',1), ('C',-1)]}, {'coeff':25, 'term':[('y1',1), ('y2',1), ('C',-1)]}]

In [18]:
Q=mult(form) #we ignore a constant term

In [19]:
Q #we are ignoring the constant term and using the fact that x^2=x, x in {0,1}

{('y2', 'y2'): -19,
 ('x1', 'x1'): -11,
 ('x2', 'x1'): 58,
 ('y2', 'x1'): 10,
 ('x2', 'x2'): -20,
 ('y2', 'x2'): 8,
 ('y1', 'y1'): -15,
 ('y1', 'x1'): 18,
 ('y2', 'y1'): 50}

# Real network example

In [20]:
form=[0]*(n+1)

for i in range(n+1):
    form[i]=[0]*(n+1)

In [21]:
B=np.random.rand(N+1,1)

In [22]:
form=[]
ys=set()

In [23]:
for e in edges:
    x,y,w=e
    
    #if x>N and y>N:
    #    continue
    
    
    term=[]
    coeff=0
    
    for i in range(1,N+1):
        for j in range(N+1,N+n+1):
            if i!=j:
                                 
                if (x,y,i,j) in edges_to_path:
                    coeff=1.0/w
                    name='y'+'_'+str(i)+'_'+str(j)
                    term.append((name,B[i].item()))
                    ys.add(name)
            
            
    if coeff!=0:
        print('x,y',x,y,term)
        form.append({'coeff':coeff, 'term':term})                

x,y 5 15 [('y_2_15', 0.9702581136106062), ('y_4_15', 0.7711614019134165), ('y_5_15', 0.0556162295890712), ('y_10_15', 0.09870769915603161)]
x,y 5 2 [('y_5_12', 0.0556162295890712), ('y_10_12', 0.09870769915603161)]
x,y 9 13 [('y_2_13', 0.9702581136106062), ('y_7_13', 0.42334467808794196), ('y_8_13', 0.48966309731127944), ('y_9_13', 0.48132141605488454), ('y_10_13', 0.09870769915603161)]
x,y 3 2 [('y_3_12', 0.6566680195844882)]
x,y 1 13 [('y_1_12', 0.6224253720480623), ('y_1_13', 0.6224253720480623)]
x,y 2 5 [('y_2_11', 0.9702581136106062), ('y_2_15', 0.9702581136106062)]
x,y 5 13 [('y_5_13', 0.0556162295890712)]
x,y 11 15 [('y_3_15', 0.6566680195844882), ('y_7_15', 0.42334467808794196), ('y_8_15', 0.48966309731127944), ('y_9_15', 0.48132141605488454)]
x,y 2 12 [('y_1_12', 0.6224253720480623), ('y_2_12', 0.9702581136106062), ('y_3_12', 0.6566680195844882), ('y_5_12', 0.0556162295890712), ('y_7_12', 0.42334467808794196), ('y_8_12', 0.48966309731127944), ('y_9_12', 0.48132141605488454), (

In [24]:
len(ys)

50

In [25]:
edges

{(1, 11, 0.8550095815477146),
 (1, 13, 0.13651902276258543),
 (1, 14, 0.4552114523664397),
 (1, 15, 0.7193580942721072),
 (2, 3, 0.3358722859296459),
 (2, 4, 0.8599331292728976),
 (2, 5, 0.24832691505262272),
 (2, 6, 0.4303409705286523),
 (2, 7, 0.8600341979033854),
 (2, 8, 0.5710015751835147),
 (2, 9, 0.24209810441616286),
 (2, 10, 0.9842410387580762),
 (2, 11, 0.9559853575911531),
 (2, 12, 0.38665511612196435),
 (2, 13, 0.7675423843816244),
 (3, 2, 0.3358722859296459),
 (3, 4, 0.3671880920795716),
 (3, 7, 0.6877173709908387),
 (3, 9, 0.33828160798887774),
 (3, 13, 0.7211358616091744),
 (3, 14, 0.6729130828832756),
 (3, 15, 0.7713415032141654),
 (4, 2, 0.8599331292728976),
 (4, 3, 0.3671880920795716),
 (4, 5, 0.2688155114523566),
 (4, 6, 0.10372897412869164),
 (4, 7, 0.8126127174604049),
 (4, 8, 0.9899069494890694),
 (4, 9, 0.7434852828685689),
 (4, 10, 0.9561471020946433),
 (4, 11, 0.574548906236377),
 (4, 12, 0.6756606494147235),
 (4, 13, 0.8178947166091707),
 (5, 2, 0.2483269150526

In [26]:
ys

{'y_10_11',
 'y_10_12',
 'y_10_13',
 'y_10_14',
 'y_10_15',
 'y_1_11',
 'y_1_12',
 'y_1_13',
 'y_1_14',
 'y_1_15',
 'y_2_11',
 'y_2_12',
 'y_2_13',
 'y_2_14',
 'y_2_15',
 'y_3_11',
 'y_3_12',
 'y_3_13',
 'y_3_14',
 'y_3_15',
 'y_4_11',
 'y_4_12',
 'y_4_13',
 'y_4_14',
 'y_4_15',
 'y_5_11',
 'y_5_12',
 'y_5_13',
 'y_5_14',
 'y_5_15',
 'y_6_11',
 'y_6_12',
 'y_6_13',
 'y_6_14',
 'y_6_15',
 'y_7_11',
 'y_7_12',
 'y_7_13',
 'y_7_14',
 'y_7_15',
 'y_8_11',
 'y_8_12',
 'y_8_13',
 'y_8_14',
 'y_8_15',
 'y_9_11',
 'y_9_12',
 'y_9_13',
 'y_9_14',
 'y_9_15'}

In [27]:
form

[{'coeff': 6.172789246552571,
  'term': [('y_2_15', 0.9702581136106062),
   ('y_4_15', 0.7711614019134165),
   ('y_5_15', 0.0556162295890712),
   ('y_10_15', 0.09870769915603161)]},
 {'coeff': 4.026949715813491,
  'term': [('y_5_12', 0.0556162295890712), ('y_10_12', 0.09870769915603161)]},
 {'coeff': 2.097736383487256,
  'term': [('y_2_13', 0.9702581136106062),
   ('y_7_13', 0.42334467808794196),
   ('y_8_13', 0.48966309731127944),
   ('y_9_13', 0.48132141605488454),
   ('y_10_13', 0.09870769915603161)]},
 {'coeff': 2.9773221605115308, 'term': [('y_3_12', 0.6566680195844882)]},
 {'coeff': 7.324986509309099,
  'term': [('y_1_12', 0.6224253720480623), ('y_1_13', 0.6224253720480623)]},
 {'coeff': 4.026949715813491,
  'term': [('y_2_11', 0.9702581136106062), ('y_2_15', 0.9702581136106062)]},
 {'coeff': 1.1707679390371537, 'term': [('y_5_13', 0.0556162295890712)]},
 {'coeff': 11.246049231475288,
  'term': [('y_3_15', 0.6566680195844882),
   ('y_7_15', 0.42334467808794196),
   ('y_8_15', 0.4

In [28]:
for i in range(1,N+1):
    
    term=[]
    
    for j in range(N+1,n+N+1):

        name='y'+'_'+str(i)+'_'+str(j)

        if name in ys:
            term.append((name,1))


    term.append(('C',-1))

    print('term',term)

           
    form.append({'coeff':V, 'term':term})   

                    
                    

term [('y_1_11', 1), ('y_1_12', 1), ('y_1_13', 1), ('y_1_14', 1), ('y_1_15', 1), ('C', -1)]
term [('y_2_11', 1), ('y_2_12', 1), ('y_2_13', 1), ('y_2_14', 1), ('y_2_15', 1), ('C', -1)]
term [('y_3_11', 1), ('y_3_12', 1), ('y_3_13', 1), ('y_3_14', 1), ('y_3_15', 1), ('C', -1)]
term [('y_4_11', 1), ('y_4_12', 1), ('y_4_13', 1), ('y_4_14', 1), ('y_4_15', 1), ('C', -1)]
term [('y_5_11', 1), ('y_5_12', 1), ('y_5_13', 1), ('y_5_14', 1), ('y_5_15', 1), ('C', -1)]
term [('y_6_11', 1), ('y_6_12', 1), ('y_6_13', 1), ('y_6_14', 1), ('y_6_15', 1), ('C', -1)]
term [('y_7_11', 1), ('y_7_12', 1), ('y_7_13', 1), ('y_7_14', 1), ('y_7_15', 1), ('C', -1)]
term [('y_8_11', 1), ('y_8_12', 1), ('y_8_13', 1), ('y_8_14', 1), ('y_8_15', 1), ('C', -1)]
term [('y_9_11', 1), ('y_9_12', 1), ('y_9_13', 1), ('y_9_14', 1), ('y_9_15', 1), ('C', -1)]
term [('y_10_11', 1), ('y_10_12', 1), ('y_10_13', 1), ('y_10_14', 1), ('y_10_15', 1), ('C', -1)]


In [29]:
form

[{'coeff': 6.172789246552571,
  'term': [('y_2_15', 0.9702581136106062),
   ('y_4_15', 0.7711614019134165),
   ('y_5_15', 0.0556162295890712),
   ('y_10_15', 0.09870769915603161)]},
 {'coeff': 4.026949715813491,
  'term': [('y_5_12', 0.0556162295890712), ('y_10_12', 0.09870769915603161)]},
 {'coeff': 2.097736383487256,
  'term': [('y_2_13', 0.9702581136106062),
   ('y_7_13', 0.42334467808794196),
   ('y_8_13', 0.48966309731127944),
   ('y_9_13', 0.48132141605488454),
   ('y_10_13', 0.09870769915603161)]},
 {'coeff': 2.9773221605115308, 'term': [('y_3_12', 0.6566680195844882)]},
 {'coeff': 7.324986509309099,
  'term': [('y_1_12', 0.6224253720480623), ('y_1_13', 0.6224253720480623)]},
 {'coeff': 4.026949715813491,
  'term': [('y_2_11', 0.9702581136106062), ('y_2_15', 0.9702581136106062)]},
 {'coeff': 1.1707679390371537, 'term': [('y_5_13', 0.0556162295890712)]},
 {'coeff': 11.246049231475288,
  'term': [('y_3_15', 0.6566680195844882),
   ('y_7_15', 0.42334467808794196),
   ('y_8_15', 0.4

In [30]:
Q=mult(form)

In [31]:
Q

{('y_2_15', 'y_2_15'): -9990.397957509362,
 ('y_4_15', 'y_2_15'): 9.237277963125557,
 ('y_5_15', 'y_2_15'): 0.6661933166008486,
 ('y_2_15', 'y_10_15'): 1.1823600765578914,
 ('y_4_15', 'y_4_15'): -9994.11684414359,
 ('y_5_15', 'y_4_15'): 0.5294906218959374,
 ('y_4_15', 'y_10_15'): 0.9397400974177958,
 ('y_5_15', 'y_5_15'): -9999.98090654439,
 ('y_5_15', 'y_10_15'): 0.0677741402543801,
 ('y_10_15', 'y_10_15'): -9999.894157227982,
 ('y_5_12', 'y_5_12'): -9999.979544176498,
 ('y_5_12', 'y_10_12'): 0.07261000205012409,
 ('y_10_12', 'y_10_12'): -9999.889865883577,
 ('y_2_13', 'y_2_13'): -9994.136679659698,
 ('y_7_13', 'y_2_13'): 1.7233055795255667,
 ('y_8_13', 'y_2_13'): 1.9932674044599814,
 ('y_9_13', 'y_2_13'): 1.9593109935356823,
 ('y_2_13', 'y_10_13'): 0.4018085912075279,
 ('y_7_13', 'y_7_13'): -9989.673854224826,
 ('y_8_13', 'y_7_13'): 23.88754499712352,
 ('y_9_13', 'y_7_13'): 0.8548899207303284,
 ('y_7_13', 'y_10_13'): 4.815320203011595,
 ('y_8_13', 'y_8_13'): -9984.348167322547,
 ('y_

In [32]:
len(Q)

268

In [33]:
sampler = EmbeddingComposite(DWaveSampler())

In [34]:
response=sampler.sample_qubo(Q, num_reads=1)

In [35]:
for data in response.data(['sample','energy', 'num_occurrences']):
    
    print(data.sample, 'energy', data.energy, 'num_occurrences', data.num_occurrences) 

{'y_10_11': 0, 'y_10_12': 0, 'y_10_13': 0, 'y_10_14': 0, 'y_10_15': 1, 'y_1_11': 0, 'y_1_12': 1, 'y_1_13': 0, 'y_1_14': 0, 'y_1_15': 0, 'y_2_11': 0, 'y_2_12': 0, 'y_2_13': 0, 'y_2_14': 0, 'y_2_15': 1, 'y_3_11': 0, 'y_3_12': 0, 'y_3_13': 1, 'y_3_14': 0, 'y_3_15': 0, 'y_4_11': 0, 'y_4_12': 0, 'y_4_13': 1, 'y_4_14': 0, 'y_4_15': 0, 'y_5_11': 0, 'y_5_12': 1, 'y_5_13': 0, 'y_5_14': 0, 'y_5_15': 0, 'y_6_11': 0, 'y_6_12': 0, 'y_6_13': 0, 'y_6_14': 0, 'y_6_15': 1, 'y_7_11': 1, 'y_7_12': 0, 'y_7_13': 0, 'y_7_14': 0, 'y_7_15': 0, 'y_8_11': 0, 'y_8_12': 0, 'y_8_13': 0, 'y_8_14': 1, 'y_8_15': 0, 'y_9_11': 1, 'y_9_12': 0, 'y_9_13': 0, 'y_9_14': 0, 'y_9_15': 0} energy -99965.89284235513 num_occurrences 1


# Saving function f

In [48]:
func_f=[0]*(N+1)

In [49]:
for i in range(1,N+1):
    for j in range(N+1,n+N+1):
        name='y_'+str(i)+'_'+str(j)
        #print(data.sample[name])
        
        if data.sample[name]:
            func_f[i]=j
            break
            
    #if not func_f[i]:
    #    func_f[i]=N+1

In [50]:
with open('test/'+file_best_clustering, 'w') as f:
    
    f.write('{} {}\n\n'.format(n,N))

    for i in range(1,N+1):
        f.write('{} {}\n'.format(i,func_f[i]))
        
                    
    f.close()

In [51]:
func_f[1:]

[12, 15, 13, 13, 12, 15, 11, 14, 11, 15]